In [1]:
from brian2 import *
#from brian2tools import *
import numpy as np
%matplotlib inline


In [2]:
# SIMULATION PARAMETERS

prefs.codegen.target = 'numpy'
# the numerical resolution method that shall be used by Brian
diff_method = "euler"
time_step = 0.01 * ms
defaultclock.dt = time_step 

In [3]:
################################## NEURONS AND SYNAPSES PARAMETERS ################################## 

######### (A) Neuronal parameters, used in (1) and (4) ########

# time constants
tau_m = 1.0 * ms # ok
tau_m_inh = 0.5 * ms # ok
# membrane potential after reset
v_rest = 0.0 # ok
# spiking threshold
theta_u = 0.5 # ok
theta_inh = 0.01 # ok

########## (B) Synaptic parameters, used in (2) and (3) for different synapse types ######### 

# temporal layer to som layer (u to v)
tau_r_afferent = 0.2 * ms
tau_f_afferent = 1.0 * ms

# temporal layer (u to inh exc, u to inh inh, inh to u)
tau_r_exc = 0.4 * ms
tau_f_exc = 2.0 * ms
tau_r_inh = 0.2 * ms
tau_f_inh = 1.0 * ms
tau_r_inh2u = 1.0 * ms
tau_f_inh2u = 5.0 * ms

########## (C) Maximum magnitudes of synaptic connection strength ##########
w_syn_u2inh_exc_max = 1.0
w_syn_u2inh_inh_max = 1.0
w_syn_inh2u_max = 100.0

In [89]:
class ReceptiveField:
    # Parameter used in standard deviation definition
    gamma = 1.5

    def __init__(self, bank_size = 10, I_min =0.05 , I_max =0.95 ):
        # number of units used to encode a real value  
        self.bank_size = bank_size
        # field_mu corresponds to the mean of the gaussian and shall be a numpy array 
        self.field_mu = np.zeros(bank_size)
        for i in range (bank_size): 
            mui = I_min + ((2*(i+1)-2)/2*(I_max - I_min)/(bank_size))
            self.field_mu[i] = mui
        # field_sigma corresponds to the standard deviation of the gaussian and shall be a float
        self.field_sigma = 1/self.gamma * (I_max - I_min)
        
    def float_to_potential(self, input_vector):
        try:
            input_vector = input_vector.reshape((input_vector.shape[0], 1))
            #self.field_mu = self.field_mu.reshape((self.field_mu.shape[0], 1))
            
        except Exception as exc:
            print("Exception: {0}\nObject shape: {1}".format(repr(exc), input_vector.shape))
            exit(1)
            
        #résultat de f(x)
        result = e**(-((input_vector - self.field_mu)**2) /2*self.field_sigma)
        return result

In [99]:
rf = ReceptiveField()
inp_vector = np.random.uniform(0,1,10)
inp_vector

array([0.89759576, 0.17823931, 0.63985298, 0.59922281, 0.37226606,
       0.89083319, 0.2839826 , 0.24400128, 0.44100763, 0.41056132])

In [100]:
rf.field_sigma  # 0.59999999

0.5999999999999999

In [102]:
inp_poentials = rf.float_to_potential(inp_vector) # il me semble que ça devait un tableau genre gaussienne avec 1 au milieu 
inp_poentials

array([[0.80611766, 0.84182309, 0.87484791, 0.90476046, 0.93115932,
        0.95368227, 0.97201451, 0.98589603, 0.99512771, 0.99957606],
       [0.99507855, 0.99956142, 0.99919657, 0.99398931, 0.98401523,
        0.96941839, 0.95040786, 0.92725271, 0.90027571, 0.86984583],
       [0.90088473, 0.9277844 , 0.95085487, 0.96977443, 0.98427523,
        0.99414952, 0.99925468, 0.99951658, 0.99493141, 0.98556577],
       [0.91348008, 0.93869405, 0.95992738, 0.9768818 , 0.9893159 ,
        0.99705081, 0.99997448, 0.99804443, 0.99128871, 0.97980528],
       [0.96932374, 0.983946  , 0.99394651, 0.99918081, 0.99957294,
        0.99511717, 0.98587824, 0.97198972, 0.95365083, 0.93112167],
       [0.8088837 , 0.84440323, 0.87720887, 0.90687092, 0.93299059,
        0.95520894, 0.97321507, 0.98675333, 0.99562938, 0.99971484],
       [0.98370979, 0.9938    , 0.99912615, 0.9996109 , 0.99524721,
        0.98609848, 0.97229697, 0.95404071, 0.9315887 , 0.90525481],
       [0.98877255, 0.99676038, 0.9999411

## notes séance 06/02
c'est le temps d'émission qui encode l'information 
c'est une propagation 

Tous les neurones sont les mêmes dans u et v c'est les mêmes modèles mais avec des paramètre différents 
fèche exitateur
flèche plate inhibiteur 

c'est dans le modèle des synapse qu'on mettra les fonctions alpha 
u = à la somme des poids des synapses 
w le poids synaptique

fonctions alpha les règles des synapses 

pas obligé de faire l'apprentissage dans un premier temps 

eq 5 : adapter les poids synaptiques
    --> en fonction de ce qui est reçu on applique une règle ou l'autre 
    
    
summed variable sert pour l'eq 4 

eq 6 : implémenter la différence de gaussienne 

eq 7 réduire le rayon au cours du temps
    T n'est pas donné (essayer 10 on peut jouer avec)
    n non plus n=1 on peut essayer de le faire varier 